In [73]:
import re


In [74]:
file1 = open('input.txt', 'r')
lines = file1.readlines()


In [75]:
seeds = []
seed_groups = []
maps = {}
current_map = ()
for line in lines:
    if re.match(r'seeds:', line) is not None:
        seeds_output = re.findall(r'\b\d+\b', line)
        seeds = seeds_output
        for i in range(0, len(seeds_output), 2):
            seed_groups.append((int(seeds_output[i]), int(seeds_output[i+1])))
    if line == "\n" or not line:
        continue

    if re.match(r'(\w*)-to-(\w*) map', line) is not None:
        current_map =re.match(r'(\w*)-to-(\w*) map', line).groups()
        maps[f"{current_map[0]}-to-{current_map[1]}"] = {}
        maps[f"{current_map[1]}-to-{current_map[0]}"] = {}
        continue

    if re.match(r'^\d+ \d+ \d+$', line) is not None:
        (elem1, elem2, cnt) = re.match(r'^(\d+) (\d+) (\d+)$', line).groups()
        maps[f"{current_map[0]}-to-{current_map[1]}"][int(elem2)] = {"diff": int(elem1) - int(elem2), "cnt": int(cnt)}
        maps[f"{current_map[1]}-to-{current_map[0]}"][int(elem1)] = {"diff": int(elem2) - int(elem1), "cnt": int(cnt)}


In [76]:
def get_value(dict, key):
    key_under = max([i for i in list(dict.keys()) if i <= key], default=None)
    if key_under is None:
        return key
    if dict[key_under]["cnt"] + key_under < key:
        return key
    return key+dict[key_under]["diff"]


In [77]:
min_value = 999999999999999999999999999999999999
seed_details = {}

for seed in seeds:
    soil = get_value(maps["seed-to-soil"], int(seed))
    fertilizer = get_value(maps["soil-to-fertilizer"], soil)
    water = get_value(maps["fertilizer-to-water"], fertilizer)
    light = get_value(maps["water-to-light"], water)
    temperature = get_value(maps["light-to-temperature"], light)
    humidity = get_value(maps["temperature-to-humidity"], temperature)
    location = get_value(maps["humidity-to-location"], humidity)
    if location < min_value:
        min_value = location
    seed_details[seed] = {
        "seed": seed,
        "soil": soil,
        "fertilizer": fertilizer,
        "water": water,
        "light": light,
        "temperature": temperature,
        "humidity": humidity,
        "location": location
    }
print(f"part1: {min_value}")


part1: 35


In [78]:
current_location = 219000000 # starting here to save time, but did multiple runs to find the right starting point
found = False
while not found:
    if current_location % 1000000 ==0:
        print(f"location:{current_location}")
    humidity = get_value(maps["location-to-humidity"], current_location)
    temperature = get_value(maps["humidity-to-temperature"], humidity)
    light = get_value(maps["temperature-to-light"], temperature)
    water = get_value(maps["light-to-water"], light)
    fertilizer = get_value(maps["water-to-fertilizer"], water)
    soil = get_value(maps["fertilizer-to-soil"], fertilizer)
    seed = get_value(maps["soil-to-seed"], soil)
    for seed_tupple in seed_groups:
        if seed_tupple[0] <= seed <= seed_tupple[0] + seed_tupple[1]:
            print(f"part2 location:{current_location} seed:{seed}")
            found = True
            break

    current_location += 1



location:0
part2 location:46 seed:82


In [79]:
from multiprocessing.pool import ThreadPool as Pool
from multiprocessing import Value


pool_size = 20
increment = 340000000/20

# define worker function before a Pool is instantiated
def worker(entry):
    current_location = entry * increment
    found = False
    seed = 0
    initial_location = current_location

    while not found and current_location<=initial_location+increment:
        humidity = get_value(maps["location-to-humidity"], current_location)
        temperature = get_value(maps["humidity-to-temperature"], humidity)
        light = get_value(maps["temperature-to-light"], temperature)
        water = get_value(maps["light-to-water"], light)
        fertilizer = get_value(maps["water-to-fertilizer"], water)
        soil = get_value(maps["fertilizer-to-soil"], fertilizer)
        seed = get_value(maps["soil-to-seed"], soil)
        for seed_tupple in seed_groups:
            if seed_tupple[0] <= seed <= seed_tupple[0] + seed_tupple[1]:
                return current_location

        current_location += 1

pool = Pool(processes=pool_size)

variable = Value('i', 0)

pool = multiprocess.Pool(processes = pool_size)
output_from_processes = pool.map(worker, range(pool_size))


print(f"part 2: min location {min(value for value in output_from_processes if value is not None)}")


part 2: min location 46
